In [12]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

In [13]:
bd.projects

Brightway2 projects manager with 8 objects:
	advlca23
	advlca25
	bc24-hackathon-bike-project
	bike_example
	datapackage
	default
	ecoinvent_import
	ecoinvent_import_from_local
Use `projects.report()` to get a report on all projects.

In [14]:
bi.remote.install_project("ecoinvent-3.10-biosphere", "bc24-hackathon-bike-project", overwrite_existing=True)

Restoring project backup archive - this could take a few minutes...
Restored project: bc24-hackathon-bike-project


'bc24-hackathon-bike-project'

In [15]:
bd.projects.set_current('bc24-hackathon-bike-project')
bd.databases

Databases dictionary with 1 object(s):
	ecoinvent-3.10-biosphere

In [6]:
len(bd.methods)

668

In [7]:
bd.methods.random()

('ReCiPe 2016 v1.03, midpoint (I)',
 'particulate matter formation',
 'particulate matter formation potential (PMFP)')

In [16]:
db = bd.Database('bike_production_example')
db.register()

In [17]:
bd.databases

Databases dictionary with 2 object(s):
	bike_production_example
	ecoinvent-3.10-biosphere

In [18]:
# First activity: the bike
data = {
    'code': 'bike',
    'name': 'bike production',
    'location': 'DK',
    'unit': 'bike'
}
bike = db.new_node(**data)
bike.save()



# First input
data = {
    'code': 'ng',
    'name': 'natural gas production',
    'location': 'NO',
    'unit': 'MJ'
}

ng = db.new_node(**data)
ng.save()

# Second input
data = {
    'code': 'cf',
    'name': 'carbon fibre production',
    'location': 'DE',
    'unit': 'kg'
}

cf = db.new_node(**data)
cf.save()

# We add the exchange to the bike activity, from the carbon fiber prod

bike.new_edge(
    amount=2.5, 
    type='technosphere',
    input=cf
).save()

In [19]:
# Verify by printing the exchanges of the bike
a_bike = db.get('bike')
for e in a_bike.exchanges():
    print(e)


# Add another exchange to the carbon fiber production
cf.new_edge(
    amount=237.3,  # plus 58 kWh of electricity, in ecoinvent 3.8 
    uncertainty_type=5, 
    minimum=200, 
    maximum=300, 
    type='technosphere',
    input=ng,
).save()

Exchange: 2.5 kg 'carbon fibre production' (kg, DE, None) to 'bike production' (bike, DK, None)>


In [20]:
for act in db:
    print(act)

'bike production' (bike, DK, None)
'carbon fibre production' (kg, DE, None)
'natural gas production' (MJ, NO, None)


In [21]:
# Let's find the co2 emission to air from the biosphere tha tis used 
# in the EF V3.0 method:

# This assumes "namespaced" methods in the project
random_m = bd.methods.random()
if len(random_m) == 4:
    # we have namespaced methods
    ef_cc_key = ('ecoinvent-3.10', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')
else:
    ef_cc_key = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
ef_cc_method = bd.Method(ef_cc_key)
ef_cc_method.metadata

# Here are the actual characterization factors of the method
cfs = ef_cc_method.load()

bio_db = bd.Database('biosphere3')
for a_cf in cfs:
    # a_cf: under bw25, this would be a tuple with flow id and cf
    flow = bd.get_activity(a_cf[0])
    if 'Carbon dioxide, fossil' in flow['name'] and ('air',) == flow['categories']:
        # print(flow)
        # print(flow_key)
        break
print(f"Found flow: {flow}")


Found flow: 'Carbon dioxide, fossil' (kilogram, None, ('air',))


In [22]:
cf.new_edge(
    amount=26.6, 
    uncertainty_type=5, 
    minimum=26,
    maximum=27.2, 
    type='biosphere',
    input=flow,
).save()

In [23]:
# Now we can do a regular LCA, with an actual lcia method

# printing the contribution analysis
import bw2analyzer as bwa

bwa.print_recursive_supply_chain(bike)

lca = bc.LCA({bike:1}, ef_cc_key)

lca.lci()
lca.lcia()
lca.score

1: 'bike production' (bike, DK, None)
  2.5: 'carbon fibre production' (kg, DE, None)
    593: 'natural gas production' (MJ, NO, None)


66.50000095367433

In [24]:
ef_cc_m = bd.Method(ef_cc_key)
print(f"{ef_cc_m} - score for: {bike} => {lca.score} {ef_cc_m.metadata['unit']}")

Brightway2 Method: EF v3.1: climate change: global warming potential (GWP100) - score for: 'bike production' (bike, DK, None) => 66.50000095367433 kg CO2-Eq


In [25]:
bwa.print_recursive_calculation(bike, ef_cc_key)

Fraction of score | Absolute score | Amount | Activity
0001 |  66.5 |     1 | 'bike production' (bike, DK, None)
  0001 |  66.5 |   2.5 | 'carbon fibre production' (kg, DE, None)
